In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import re

## 데이터 concat
- 필요하신 분들만 진행해주세요!

In [ ]:
import pandas as pd

keyword = ['1', '2', '3', '4', '5', '6', '7', '8']
data_list = []  # 데이터를 임시로 저장할 리스트

# 각 키워드에 해당하는 데이터를 불러와서 리스트에 추가
for i in keyword:
    data = pd.read_csv(f"레몬테라스_공간_{i}.csv")
    data_list.append(data)

# 리스트를 한 번에 합치기
df = pd.concat(data_list, axis=0).reset_index(drop=True)

df

## concat 필요하지 않으신 분들은 여기서부터 진행해주세요!!
- 통합본이 따로 있는 경우

In [ ]:
df = pd.read_csv("효녀로청_공간.csv", index_col = 0)

df.head()

## 중복제거

In [ ]:
df = df.drop_duplicates(subset='url',keep='first').reset_index(drop=True)
df

## 날짜기준 필터링

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d- %H:%M', errors='coerce') #네이버 카페

# NaT 확인 (잘못된 값)
invalid_dates = df[df['date'].isna()]
print("잘못된 날짜 형식의 데이터:")
print(invalid_dates)

# 기준 날짜 이후의 데이터 필터링
df = df[df['date'] >= pd.to_datetime('2020-01-01')]

df

In [ ]:
df['contents'] = df['contents'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df['contents'] = df['contents'].str.replace(r'\s+', ' ', regex=True)

df['comments'] = df['comments'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df['comments'] = df['comments'].str.replace(r'\s+', ' ', regex=True)
# 결과 확인
df.head()

## \n 제거

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<[^>]+>', '', text)
        text = re.sub(r'\n', ' ', text)
        text = re.sub(r'\s+', ' ', text)

    return text

In [ ]:
df['title'] = df['title'].apply(preprocess_text)
df['contents'] = df['contents'].apply(preprocess_text)
df['comments'] = df['comments'].apply(preprocess_text)
df['date'] = df['date'].apply(preprocess_text)

## 제목에서 광고글 제거

In [ ]:
df1 = df.copy()

In [ ]:
words_to_delete = ['협찬', '체험단', '무료체험', '예약링크', '홍보','무료제공','무상제공','위스토어', '출석체크','체험단','긴급','가입인사','모집','양도매매','판매', '산후조리원 후기', '산후조리원후기', '임테기',
                   '레시피', '출산후기']

pattern = '|'.join(words_to_delete)

delete = df1['title'].str.contains(pattern)

In [ ]:
#삭제할 행 확인
df1['title'][df1.index[delete]]

In [ ]:
df1.drop(df1[delete].index, inplace = True)
df2= df1.reset_index(drop=True)
df2

## 네이버 카페만 댓글 추가 작업 필요

In [ ]:
# comments 열 데이터를 분리하여 리스트로 변환
df2['comments'] = df2['comments'].apply(
    lambda x: [comment.strip() for comment in str(x).split('"') if comment.strip()]
)

# 결과 확인
df2.comments[0]

## 10글자 미만 댓글 제거

In [ ]:
# 댓글 리스트에서 10글자 미만의 요소를 제거하는 함수
def filter_short_comments(comment_list):
    # 리스트인지 확인 후 각 요소의 길이를 확인해 10글자 이상인 요소만 남김
    if isinstance(comment_list, list):
        return [comment for comment in comment_list if len(comment) >= 10]
    return comment_list  # 리스트가 아니면 그대로 반환

# DataFrame의 comments 열에 적용
df2['comments'] = df2['comments'].apply(filter_short_comments)

# comments 열이 빈 리스트인 행 제거
df2['comments'] = df2['comments'].apply(lambda x: '댓글 없음' if isinstance(x, list) and len(x) == 0 else ' '.join(x) if isinstance(x, list) else str(x))

df2

## 본문 10글자 미만 제거

In [ ]:
# 10글자 미만인 행 제거
df2 = df2[df2['contents'].str.len() >= 10]

# 결과 출력
df2

In [ ]:
df3 = df2.copy()

In [ ]:
df3['contents'] = df3['contents'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df3['contents'] = df['contents'].str.replace(r'\s+', ' ', regex=True)

df3['comments'] = df3['comments'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
df3['comments'] = df3['comments'].str.replace(r'\s+', ' ', regex=True)
# 결과 확인
df3

In [ ]:
# 10글자 미만인 행 제거
df3 = df3[df3['contents'].str.len() >= 10]

# 결과 출력
df3

## 게시판 기준 필터링

In [ ]:
df3['board_titles'] = df3['board_titles'].apply(lambda x: x.split('\n')[0])

In [ ]:
import pandas as pd

# pandas 출력 설정 변경
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 해제
a = []
# 모든 고유 값 출력
unique_titles = df3.board_titles.unique()
for i in unique_titles:
    a.append(i)

print(a)

### 게시판 확인!!
- 게시판 unique값 확인하시고, 판매 혹은 이벤트와 같은 맥락의 게시판 이름이 있으면 board_list에 넣고 해당 데이터 삭제 진행해주세요!
- 예시는 레몬테라스 기준으로 되어있는 겁니다!!

In [ ]:
board_list = {'임신&출산 후기', '돌/백일잔치 후기', '난임/인공/시험관 질문방', '돌/백일잔치 궁금증', '▶이벤트 후기', '알뜰 벼룩 - 아이용품', '쇼핑할인 정보방', '유산관련 질문방', '규정위반글-삭제예정', '알뜰 벼룩 - 기타생활용품','테스터 질문방',
 '산후다이어트 질문방', '💖유교전 유아교육전 (11.14~17,코엑스)', '🚩 추천인 요기요','🛒 맘스홀릭 공구', '분만 질문방','🎀 46회 베페 베이비페어(9.26~29, 코엑스)', '코로나 치료 경험담', '알뜰 벼룩 - 엄마용품', '맘스홀릭 맘스토어', 'D라인 뽐내기',
 '아가에게 쓰는 편지', '카페이용 문의', '출석덧글 [끝말잇기]', '약사 상담 - 임신, 여성 건강 프리솔라', '🌈 맘스홀릭베이비 페어','✅ 출산준비 리스트', '국민청원 공유하기', '돌/백일/산후조리 공구-매주(화)', '초음파 질문방',
 '요리조리 레시피','가입 인사', '맘스홀릭 온정나눔', '맘스홀릭 클래스', '🍓 이유식준비 리스트','규정위반(벼룩)-삭제예정', '[종료] 강릉 세인트존스호텔', '[종료] 렌탈몬스터', '🎉 이벤트 공지 ⭐', '★이벤트 당첨자발표',
 '[종료] LG전자 공식 구독(렌탈) 서비스', '＊ 4월 온라인 페어 종료', '팸퍼스 터치오브네이처 체험단 (11/25)', '미사용게시판', '팸퍼스 터치오브네이처 체험단 (12/2)', '[종료] 여성건강주치의 인애한의원', '공지사항',
 '[종료] 법률 상담 - 로펌 청하', '[종료] 가슴 성형 상담 - 에이비 성형외과', '미사용게시판 50', '미사용게시판49', '(종료)종합법률무료상담 풍경', '치아교정 무턱교정 일리노이치과', '이태리 프리미엄 유모차, 잉글레시나', 
 '[종료] 맘스퀸', '[종료] 법률상담- YK법무법인', '미사용게시판 42', '[종료] 수인재 한의원', '(경기) 맘스홀릭 맘스토어', '📊 맘스홀릭 리서치', '[종료]우리 함께 행복 하기스', '엄마들의 패션 트렌드', '제천맘들~', '(종료) 여성건강주치의 인애한의원'}

# board_titles가 board_list에 없는 행만 필터링
df3 = df3[~df3['board_titles'].isin(board_list)].reset_index(drop=True)

In [ ]:
df3.shape

## 제목, 본문, 댓글을 하나로 통합

In [ ]:
# 혹시 리뷰 중에 Null 값을 가진 샘플이 있는지 확인
print(df3.isnull().sum())

In [ ]:
df3 = df3.dropna()

In [ ]:
# 위에서 Null값이 나오면 공백으로 수정 (Null 값 포함해서 doc로 합치면 Null값으로 반환됨)
df3['contents'] = df3['contents'].fillna('')

print(df3.isnull().values.any()) # 제거되었는지 확인

In [ ]:
df3.loc[:,'review'] = df3['title'] + ' ' + df3['contents'] + ' ' + df3['comments']

In [ ]:
df3['review']

In [ ]:
df3 = df3.reset_index(drop=True)

In [ ]:
df_doc = df3.review

In [ ]:
df_doc

## 반복되는 문장 삭제

In [ ]:
# 따옴표 안에 삭제할 문구를 복사해서 붙여넣기 해준다.
df_doc = df_doc.str.replace("경상도 3pl 전문 라온아토입니다","")
df_doc = df_doc.str.replace("(원스톱 사업자등록) 월 18000원 비상주 서비스 바로 가입! 바로 임대차계약서 발급까지!","")
df_doc = df_doc.str.replace("출석합니다","")
df_doc = df_doc.str.replace("출첵합니다.","")
df_doc = df_doc.str.replace("가입인사","")
df_doc = df_doc.str.replace("잘부탁드립니다.","")
df_doc = df_doc.str.replace("감사합니다.","")
df_doc = df_doc.str.replace("부탁드립니다","")
df_doc = df_doc.str.replace("연락부탁드립니다.","")

df_doc = df_doc.str.replace("긴글+ 사진폭탄 주의","")
df_doc = df_doc.str.replace("질문 있습니당.","")
df_doc = df_doc.str.replace("★","")

df_doc = df_doc.str.replace("댓글 없음","")
df_doc = df_doc.str.replace("< 아래 양식을 지켜주셔야 글 삭제와 회원강등이 안되니 꼭 지켜서 작성해주세요 >","")
df_doc = df_doc.str.replace("양식을 무시할 경우 운영규칙상 삭제 및 활동정지 될수 있으니 참고부탁드립니다.","")
df_doc = df_doc.str.replace("인스타 양도글은 인터넷 진흥원에서 안된다고 경고를 해서 카페에서 금지하고 있습니다","")
df_doc = df_doc.str.replace("카페활동 없이 양도글만 남길 경우 사이트의 자세한 설명없이 개인연락 유도글은 제재하고 있습니다","")

df_doc = df_doc.str.replace("안녕하세요","")
df_doc = df_doc.str.replace("안녕하세요!","")
df_doc = df_doc.str.replace("안녕하세요.","")

df_doc = df_doc.str.replace("쪽지드렸습니다", "")
df_doc = df_doc.str.replace("챗드릴게요","")
df_doc = df_doc.str.replace("쪽지주세요","")
df_doc = df_doc.str.replace("쪽지드릴게요","")
df_doc = df_doc.str.replace("쪽지 드렸어요","")
df_doc = df_doc.str.replace("댓글이나 쪽지로 주세요","")

df_doc = df_doc.str.replace(" 카페 활동 공지 사항 필독 <-- 가입인사 글쓰기 후 확인하시고 활동하셔야 합니다","")
df_doc = df_doc.str.replace("◆◆◆◆◆◆    진단평가와 관련된 내용만 올려주세요 그 외의 글은 삭제 및 활동제재합니다 ◆◆◆◆◆","")
df_doc = df_doc.str.replace("구합니다.","")
df_doc = df_doc.str.replace("등업조건:가입인사,출석 게시판 각 1개씩 글을 쓰고 각 게시판에 10씩 댓글을 달면 자동등업","")
df_doc = df_doc.str.replace("판매합니다","")
df_doc = df_doc.str.replace("수고하세요.","")
df_doc = df_doc.str.replace("쪽지 부탁드려요.","")
df_doc = df_doc.str.replace("소통해요.","")
df_doc = df_doc.str.replace("소통하고싶어요.","")
df_doc = df_doc.str.replace("해드리겠습니다.","")
df_doc = df_doc.str.replace("진행해드리겠습니다.","")
df_doc = df_doc.str.replace("말씀드리겠습니다.","")
df_doc = df_doc.str.replace("모집중에있습니다.","")
df_doc = df_doc.str.replace("양도합니다.","")
df_doc = df_doc.str.replace("전매합니다.","")
df_doc = df_doc.str.replace("확인합니다","")

df_doc = df_doc.str.replace("댓글없음.","")
df_doc = df_doc.str.replace("나눔후기\)","")
df_doc = df_doc.str.replace("\[체험후기\]","")

In [ ]:
df_doc

## 텍스트 전처리

In [ ]:
df_doc = df_doc.str.replace("[0-9]+갤","개월", regex = True) # ex.8갤->개월(맞춤법 검사 시 교정 안 됌)
df_doc = df_doc.str.replace("[^가-힣a-zA-z ]"," ", regex=True) # 한글,영어만 남기고 특수문자, 숫자 삭제
df_doc = df_doc.str.replace("\s+", " ", regex = True) # white space 제거
df_doc = df_doc.str.replace(r"\\n", " ", regex=True)
df_doc = df_doc.str.replace(r"\\n", " ", regex=True)
df_doc = df_doc.str.replace(r'[^\w\s가-힣]', '', regex=True)

df_doc = df_doc.str.strip()

In [ ]:
df_doc

In [ ]:
doc = pd.DataFrame(df_doc)

In [ ]:
doc['review'] = doc['review'].str.replace(r'\[이미지[^\]]*\]', '', regex=True)
doc['review'] = doc['review'].str.replace(r'\s+', ' ', regex=True)
# 결과 확인
doc

In [ ]:
doc.to_csv("네이버카페_전처리완료.csv", encoding ='utf-8-sig', index=False)